In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

from config import api_key

import numpy as np
import requests
import json

In [90]:
# Import cities file as DataFrame
fourteen_fiftenn_pd= pd.read_csv("draft_data/2014-15.csv")

fourteen_fiftenn_pd = fourteen_fiftenn_pd.drop(columns=["Important Notes", "Type", "Electricity Production (GJ)"])

fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Facility Name'] != 'Corporate Total']
fourteen_fiftenn_pd.head(15)

,Reporting Entity,Facility Name,State,Electricity Production (Mwh),Scope 1 (t CO2-e),Scope 2 (t CO2-e),Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected,Grid,Primary Fuel
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,157700.0,19,293,312,0.00,On,NEM,Wind
1,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,ACT,59199.0,0,15,15,0.00,On,NEM,Solar
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,683834.0,77,1144,1221,0.00,On,NEM,Wind
4,AGL ENERGY LIMITED,Banimboola Hydro,VIC,38082.0,5,17,22,0.00,On,NEM,Hydro
5,AGL ENERGY LIMITED,Bayswater Power Station,NSW,13965430.0,12253123,8306,12261429,0.88,On,NEM,Black Coal
6,AGL ENERGY LIMITED,Bogong Power Station,VIC,74144.0,83,839,922,0.00,On,NEM,Hydro
7,AGL ENERGY LIMITED,Burrendong Hydro,NSW,7619.0,1,97,98,0.00,On,NEM,Hydro
8,AGL ENERGY LIMITED,Clover Hydro,VIC,42047.0,3,51,54,0.00,On,NEM,Hydro
9,AGL ENERGY LIMITED,Coopers Cogeneration,SA,25463.0,19260,86,19346,0.76,On,NEM,Gas
10,AGL ENERGY LIMITED,Copeton Hydro,NSW,38829.0,1,75,76,0.00,On,NEM,Hydro


In [84]:
# Import cities file as DataFrame
twelve_thirteen_pd = pd.read_csv("draft_data/2012-13.csv")

twelve_thirteen_pd = twelve_thirteen_pd.drop(columns=["Important Notes", "Type","Reporting Entity", "Electricity Production (GJ)", "Greenhouse Gas Emissions Scope 1 (t CO2-e)"])

twelve_thirteen_pd = twelve_thirteen_pd[twelve_thirteen_pd['Facility Name'] != 'Corporate Total']
twelve_thirteen_pd.head(15)

,Facility Name,Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e)
0,Gunning Wind Farm,297,327
1,Waubra Wind Farm,"1,095","1,189"
3,AGL Loy Yang Power Station and Mine,"259,065","19,127,315"
4,Banimboola Hydro,69,72
5,Bogong Power Station,911,911
6,Burrendong Hydro,34,34
7,Clover Hydro,"1,966","1,966"
8,Coopers Cogeneration,100,"20,201"
9,Copeton Hydro,54,54
10,Dartmouth Hydro,"2,275","2,275"


In [10]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twelve_thirteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twelve_thirteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twelve_thirteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twelve_thirteen_pd.loc[index, "lat"] = "NaN"
        twelve_thirteen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
twelve_thirteen_pd.head()



,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t CO2-e),Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e),Lat,Lng,Place_ID,Address,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"595,404",30,297,327,,,,,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,405,976",94,"1,095","1,189",,,,,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,001,380",124,"1,392","1,516",,,,,NaN,NaN
3,AGL ENERGY LIMITED,AGL Loy Yang Power Station and Mine,F,"58,566,999","18,868,250","259,065","19,127,315",,,,,-38.2537,146.576
4,AGL ENERGY LIMITED,Banimboola Hydro,F,"41,286",3,69,72,,,,,-36.5343,147.459


In [20]:
# Import cities file as DataFrame
thirteen_fourteen_pd = pd.read_csv("draft_data/2013-14.csv")
fourteen_fiftenn_pd = pd.read_csv("draft_data/2014-15.csv")
fifteen_sixteen_pd = pd.read_csv("draft_data/2015-16.csv")
sixteen_seventeen_pd = pd.read_csv("draft_data/2016-17.csv", encoding = "ISO-8859-1")
seventeen_eighteen_pd = pd.read_csv("draft_data/2017-18.csv", encoding = "ISO-8859-1")
eighteen_nineteen_pd = pd.read_csv("draft_data/2018-19.csv", encoding = "ISO-8859-1")
nineteen_twenty_pd = pd.read_csv("draft_data/2019-20.csv", encoding = "ISO-8859-1")
twenty_twentyone = pd.read_csv("draft_data/2020-21.csv", encoding = "ISO-8859-1")

In [ ]:
thirteen_fourteen_pd = thirteen_fourteen_pd[thirteen_fourteen_pd['Facility Name'] != 'Corporate Total']
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Facility Name'] != 'Corporate Total']
fifteen_sixteen_pd = fifteen_sixteen_pd[fifteen_sixteen_pd['Facility Name'] != 'Corporate Total']
sixteen_seventeen_pd = sixteen_seventeen_pd[sixteen_seventeen_pd['Facility Name'] != 'Corporate Total']
seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Facility Name'] != 'Corporate Total']
nineteen_twenty_pd = nineteen_twenty_pd[nineteen_twenty_pd['Facility Name'] != 'Corporate Total']
twenty_twentyone = twenty_twentyone[twenty_twentyone['Facility Name'] != 'Corporate Total']

In [21]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in thirteen_fourteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        thirteen_fourteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        thirteen_fourteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        thirteen_fourteen_pd.loc[index, "lat"] = "NaN"
        thirteen_fourteen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
thirteen_fourteen_pd.head()

,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t-CO2-e),Greenhouse Gas Emissions Scope 2 (t-CO2-e),Greenhouse Gas Emissions Total Emissions (t-CO2-e),Important notes,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"622,579",31,375,406,NaN,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,506,981",73,"1,173","1,246",NaN,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,129,560",104,"1,548","1,652",NaN,NaN,NaN
3,AGL ENERGY LIMITED,Banimboola Hydro,F,"94,469",16,20,36,NaN,-36.5343,147.459
4,AGL ENERGY LIMITED,Bogong Power Station,F,"356,830",83,881,964,NaN,-36.8047,147.224


In [22]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fourteen_fiftenn_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fourteen_fiftenn_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fourteen_fiftenn_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fourteen_fiftenn_pd.loc[index, "lat"] = "NaN"
        fourteen_fiftenn_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
fourteen_fiftenn_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (Mwh),Scope 1 (t CO2-e),Scope 2 (t CO2-e),Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected,Grid,Primary Fuel,Important Notes,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,567719.0,157700.0,19,293,312,0.0,On,NEM,Wind,NaN,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,F,ACT,213115.0,59199.0,0,15,15,0.0,On,NEM,Solar,NaN,-35.4898,149.143
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,2461803.0,683834.0,77,1144,1221,0.0,On,NEM,Wind,NaN,-37.3946,143.639
3,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,NaN,3242637.0,900733.0,96,1452,1548,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGL ENERGY LIMITED,Banimboola Hydro,F,VIC,137094.0,38082.0,5,17,22,0.0,On,NEM,Hydro,NaN,-36.5343,147.459


In [23]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fifteen_sixteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fifteen_sixteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fifteen_sixteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fifteen_sixteen_pd.loc[index, "lat"] = "NaN"
        fifteen_sixteen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
fifteen_sixteen_pd.head()


,Controlling corporation,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (Mwh),Total Scope 1 Emissions (t CO2-e),Total Scope 2 Emissions (t CO2-e),Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected,Grid,Primary Fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,F,NSW,"595,234","165,343",22,283,305,0.00,On,NEM,Wind,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,F,VIC,"2,352,139","653,372",93,950,"1,043",0.00,On,NEM,Wind,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Corporate Total,C,NaN,"2,947,373","818,715",115,"1,233","1,348",NaN,NaN,NaN,NaN,NaN,NaN
3,AGL ENERGY LIMITED,Banimboola Hydro,F,VIC,"170,057","47,238",0,5,5,0.00,On,NEM,Hydro,-36.5343,147.459
4,AGL ENERGY LIMITED,Bayswater Power Station,F,NSW,"64,493,748","17,914,930","15,912,788","6,252","15,919,040",0.89,On,NEM,Black Coal,-32.3926,150.949


In [24]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in sixteen_seventeen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        sixteen_seventeen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        sixteen_seventeen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        sixteen_seventeen_pd.loc[index, "lat"] = "NaN"
        sixteen_seventeen_pd.loc[index, "lng"] = "NaN"

# Visualize to confirm lat lng appear
sixteen_seventeen_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (Mwh),Total Scope 1 Emissions (t CO2-e),Total Scope 2 Emissions (t CO2-e)2,Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected2,Grid,Primary Fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,F,NSW,"562,485","156,246",46,283,329,0,On,NEM,Wind,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,F,VIC,"2,204,701","612,417",129,"1,080","1,209",0,On,NEM,Wind,-37.3946,143.639
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Corporate Total,C,-,"2,767,186","768,663",175,"1,363","1,538",-,-,-,-,NaN,NaN
3,AGL ENERGY LIMITED,Banimboola Hydro,F,VIC,"10,938","3,038",3,10,13,0,On,NEM,Hydro,-36.5343,147.459
4,AGL ENERGY LIMITED,Bayswater Power Station,F,NSW,"57,400,488","15,944,580","14,148,670","12,723","14,161,393",0.89,On,NEM,Black Coal,-32.3926,150.949


In [25]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in seventeen_eighteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        seventeen_eighteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        seventeen_eighteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        seventeen_eighteen_pd.loc[index, "lat"] = "NaN"
        seventeen_eighteen_pd.loc[index, "lng"] = "NaN"

seventeen_eighteen_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (MWh),Total Scope 1 Emissions \n(t CO2-e),Total Scope 2 Emissions \n(t CO2-e),Total Emissions \n(t CO2-e),Emission Intensity \n(t CO2-e/ MWh),Grid Connected,Grid,Primary Fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Cathedral Rocks Wind Farm,F,SA,"550,950","153,042",48,136,184,0,On,NEM,Wind,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,F,NSW,"601,740","167,150",44,205,249,0,On,NEM,Wind,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Mt Gellibrand Wind Farm,F,VIC,"4,320","1,200",0,22,22,0,On,NEM,Wind,NaN,NaN
3,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,F,VIC,"2,317,673","643,798",132,966,"1,098",0,On,NEM,Wind,-37.3946,143.639
4,ACCIONA ENERGY OCEANIA PTY. LTD.,Corporate Total,C,-,"3,474,683","965,190",224,"1,329","1,553",-,-,-,-,NaN,NaN


In [26]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in eighteen_nineteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        eighteen_nineteen_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        eighteen_nineteen_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        eighteen_nineteen_pd.loc[index, "lat"] = "NaN"
        eighteen_nineteen_pd.loc[index, "lng"] = "NaN"

eighteen_nineteen_pd.head()

,Reporting Entity,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (MWh),Total Scope 1 Emissions \n(t CO2-e),Total Scope 2 Emissions \n(t CO2-e),Total Emissions \n(t CO2-e),Emission Intensity \n(t CO2-e/ MWh),Grid Connected,Grid,Primary Fuel,lat,lng
0,Acciona Energy Oceania Pty. Ltd.,Cathedral Rocks Wind Farm,F,SA,"632,284","175,634",39,213,252,0,On,NEM,Wind,NaN,NaN
1,Acciona Energy Oceania Pty. Ltd.,Gunning Wind Farm,F,NSW,"592,807","164,669",44,195,239,0,On,NEM,Wind,-34.6906,149.427
2,Acciona Energy Oceania Pty. Ltd.,Mt Gellibrand Wind Farm,F,VIC,"844,669","234,630",36,864,900,0,On,NEM,Wind,NaN,NaN
3,Acciona Energy Oceania Pty. Ltd.,Waubra Wind Farm,F,VIC,"2,261,159","628,100",92,901,993,0,On,NEM,Wind,-37.3946,143.639
4,Acciona Energy Oceania Pty. Ltd.,Corporate Total,C,-,"4,330,919","1,203,033",211,"2,173","2,384",-,-,-,-,NaN,NaN


In [29]:
nineteen_twenty_pd.rename(columns = {'Facility name':'Facility Name'}, inplace = True)
nineteen_twenty_pd.head()

,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions (t CO2-e),Total emissions (t CO2-e),Emission intensity (t CO2-e/ MWh),Grid connected,Grid,Primary fuel
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"555,151","154,209",50,95,145,0,On,NEM,Wind
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"636,185","176,718",45,143,188,0,On,NEM,Wind
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,086,174","301,715",49,308,357,0,On,NEM,Wind
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,414,107","670,585",139,452,591,0,On,NEM,Wind
4,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,-,"4,691,617","1,303,227",283,998,"1,281",-,-,-,-


In [39]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in nineteen_twenty_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location} + {state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        nineteen_twenty_pd.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        nineteen_twenty_pd.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        nineteen_twenty_pd.loc[index, "lat"] = "NaN"
        nineteen_twenty_pd.loc[index, "lng"] = "NaN"

nineteen_twenty_pd.head()

,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions (t CO2-e),Total emissions (t CO2-e),Emission intensity (t CO2-e/ MWh),Grid connected,Grid,Primary fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"555,151","154,209",50,95,145,0,On,NEM,Wind,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"636,185","176,718",45,143,188,0,On,NEM,Wind,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,086,174","301,715",49,308,357,0,On,NEM,Wind,NaN,NaN
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,414,107","670,585",139,452,591,0,On,NEM,Wind,-37.3946,143.639
4,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,-,"4,691,617","1,303,227",283,998,"1,281",-,-,-,-,NaN,NaN


In [ ]:
twenty_twentyone.rename(columns = {'Facility name':'Facility Name'}, inplace = True)

In [42]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twenty_twentyone.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location} + {state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twenty_twentyone.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twenty_twentyone.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twenty_twentyone.loc[index, "lat"] = "NaN"
        twenty_twentyone.loc[index, "lng"] = "NaN"

twenty_twentyone.head()

,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions \n(t CO2-e),Total emissions \n(t CO2-e),Emission intensity \n(t CO2-e/ MWh),Grid connected,Grid,Primary fuel,lat,lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"512,455","142,348",47,189,236,0,On,NEM,Wind,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"575,522","159,867",48,246,294,0,On,NEM,Wind,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,018,498","282,916",60,907,967,0,On,NEM,Wind,NaN,NaN
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,339,624","649,896",158,869,"1,027",0,On,NEM,Wind,-37.3946,143.639
4,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,-,"4,446,099","1,235,027",313,"2,211","2,524",-,-,-,-,NaN,NaN


In [ ]:
thirteen_fourteen_pd = thirteen_fourteen_pd[thirteen_fourteen_pd['lat'] == 'NaN']
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['lat'] == 'NaN']
fifteen_sixteen_pd = fifteen_sixteen_pd[fifteen_sixteen_pd['lat'] == 'NaN']
sixteen_seventeen_pd = sixteen_seventeen_pd[sixteen_seventeen_pd['lat'] == 'NaN']
seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['lat'] == 'NaN']
nineteen_twenty_pd = nineteen_twenty_pd[nineteen_twenty_pd['lat'] == 'NaN']
twenty_twentyone = twenty_twentyone[twenty_twentyone['lat'] == 'NaN']

In [71]:
twenty_twentyone['Primary fuel'].value_counts()

Diesel                 123
Gas                     95
Wind                    83
Solar                   79
Landfill Gas            57
Hydro                   54
Black Coal              17
Waste Coal Mine Gas     10
Coal Seam Methane        7
Brown Coal               3
Battery                  3
Bagasse                  3
Sludge Biogas            1
Biofuel                  1
Kerosene                 1
Name: Primary fuel, dtype: int64

In [75]:
# Add year column for filtering in plots
fourteen_fiftenn_pd['Year'] = '2014-2015'

seventeen_eighteen_pd['Year'] = '2017-2018'

twenty_twentyone['Year'] = '2020-2021'


,Reporting entity,Facility Name,Type,State,Electricity production (GJ),Electricity production (MWh),Total scope 1 emissions \n(t CO2-e),Total scope 2 emissions \n(t CO2-e),Total emissions \n(t CO2-e),Emission intensity \n(t CO2-e/ MWh),Grid connected,Grid,Primary fuel,lat,lng,Year
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,F,SA,"512,455","142,348",47,189,236,0,On,NEM,Wind,NaN,NaN,2020-2021
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,NSW,"575,522","159,867",48,246,294,0,On,NEM,Wind,-34.6906,149.427,2020-2021
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,F,VIC,"1,018,498","282,916",60,907,967,0,On,NEM,Wind,NaN,NaN,2020-2021
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,VIC,"2,339,624","649,896",158,869,"1,027",0,On,NEM,Wind,-37.3946,143.639,2020-2021
5,ADANI RENEWABLE ASSET HOLDINGS PTY LTD,Adani Rugby Run Solar Farm,F,QLD,"587,856","163,293",0,533,533,0,On,NEM,Solar,NaN,NaN,2020-2021


In [68]:
wa_df2 = seventeen_eighteen_pd.loc[seventeen_eighteen_pd['State']=='WA']
wa_df2.groupby('Primary Fuel').count()

,Controlling corporation,Facility Name,Type,State,Electricity Production (GJ),Electricity Production (Mwh),Total Scope 1 Emissions (t CO2-e),Total Scope 2 Emissions (t CO2-e),Total Emissions (t CO2-e),Emission Intensity (t/Mwh),Grid Connected,Grid,lat,lng
Primary Fuel,,,,,,,,,,,,,,
Black Coal,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Diesel,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Gas,27,27,27,27,27,27,27,27,27,27,27,27,27,27
Gas/Diesel,11,11,11,11,11,11,11,11,11,11,11,11,11,11
Hydro,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Landfill Gas,5,5,5,5,5,5,5,5,5,5,5,5,5,5
Liquid Fuel,16,16,16,16,16,16,16,16,16,16,16,16,16,16
Wind,8,8,8,8,8,8,8,8,8,8,8,8,8,8
Wind/Diesel,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [43]:
wa_df3 = twenty_twentyone.loc[twenty_twentyone['State']=='WA']
wa_df3.groupby('Primary Fuel')

In [ ]:
# fourteen_fifteen, seventeen_eighteen, twenty_twentyone
# filter each df to group by state and then group by again to show primary fuel types